In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from notion_client import Client
from datetime import datetime, timedelta
import ipywidgets as widgets
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
notion = Client(auth=os.getenv('NOTION_API_SECRET'))
database_id = os.getenv('NOTION_DATABASE_ID')

In [3]:
def get_pages(next_cursor=None, date=None):
    filters = [
        {
            "property": "when",
            "date": {
                        "on_or_before": datetime.now().isoformat()
            }
        }
    ]

    if date:
        filters.append({
            "property": "when",
            "date": {
                "on_or_after": date
            }
        })

    response = notion.databases.query(
        database_id,
        start_cursor=next_cursor,
        filter={
            "and": filters
        }
    )

    pages = response['results']
    next_cursor = response.get('next_cursor')
    if next_cursor:
        pages += get_pages(next_cursor)
    return pages


pages = get_pages()

In [4]:
goal_id_map = {}

In [5]:
data = []
for page in pages:
    date = datetime.strptime(page['properties']['when']['date']['start'].split('T')[0], '%Y-%m-%d')
    date_start = page['properties']['when']['date']['start']
    date_end = page['properties']['when']['date']['end']
    if date_end is None:
        continue
    date_start = datetime.strptime(date_start, '%Y-%m-%dT%H:%M:%S.%f%z')
    date_end = datetime.strptime(date_end, '%Y-%m-%dT%H:%M:%S.%f%z')
    goal_id = page['properties']['Goal']['relation'][0]['id']
    goal = None
    
    if goal_id not in goal_id_map:
        goal_block = notion.blocks.retrieve(goal_id)
        goal = goal_block['child_page']['title']
        goal_id_map[goal_id] = goal
    else:
        goal = goal_id_map[goal_id]
    
    time_start = date_start.hour + date_start.minute/60
    time_end = date_end.hour + date_end.minute/60
    
    # if end time is before start time, split into two events
    if time_end < time_start:
        data.append({
            "date": date,
            "time_start": time_start,
            "time_end": 24,
            "goal": goal
        })
        data.append({
            "date": date + timedelta(days=1),
            "time_start": 0,
            "time_end": time_end,
            "goal": goal
        })
    else:
        data.append({
            "date": date,
            "time_start": time_start,
            "time_end": time_end,
            "goal": goal
        })
    
data = pd.DataFrame(data)
data.sort_values(by="date", inplace=True)
data

,date,time_start,time_end,goal
119,2024-01-19,0.0,15.75,Learning German
118,2024-02-06,13.5,15.50,Grad Project
117,2024-02-16,0.0,3.00,Kasandra
116,2024-02-16,19.0,20.00,Kasandra
115,2024-02-16,21.5,22.00,Kasandra
...,...,...,...,...
5,2024-03-27,0.0,0.00,Accountant
3,2024-03-27,2.0,3.50,Accountant
2,2024-03-27,4.5,5.50,Accountant
1,2024-03-27,3.5,4.50,Accountant


In [9]:
# create a heatmap for a day based on the data
def create_heatmap(start_date, end_date, goal=None):
    start_date = np.datetime64(start_date)
    end_date = np.datetime64(end_date)

    # copy data
    dataToUse = data.copy()

    if goal:  # case insensitive search
        dataToUse = dataToUse[dataToUse.goal.str.contains(goal, case=False)]

    # create a heatmap for a day
    heatmap = np.zeros((24, 7))
    heatmap_goals = np.empty((24, 7), dtype=object)
    for i in range(24):
        for j in range(7):
            item = dataToUse[(dataToUse.date.dt.weekday == j) & (dataToUse.time_start <= i) & (
                dataToUse.time_end > i) & (dataToUse.date >= start_date) & (dataToUse.date <= end_date)]
            heatmap[i, j] = item.shape[0]
            heatmap_goals[i, j] = ', '.join(item['goal'].unique().tolist())
    return heatmap, heatmap_goals


def update_heatmap(start_date, end_date, goal=None, show_annotations=False):
    heatmap, heatmap_goals = create_heatmap(start_date, end_date, goal)
    plt.figure(figsize=(10, 5))
    sns.heatmap(heatmap, xticklabels=[
                "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
                yticklabels=[
                    "12AM",
                    "01AM",
                    "02AM",
                    "03AM",
                    "04AM",
                    "05AM",
                    "06AM",
                    "07AM",
                    "08AM",
                    "09AM",
                    "10AM",
                    "11AM",
                    "12PM",
                    "01PM",
                    "02PM",
                    "03PM",
                    "04PM",
                    "05PM",
                    "06PM",
                    "07PM",
                    "08PM",
                    "09PM",
                    "10PM",
                    "11PM",
    ])

    if show_annotations:
        for i in range(heatmap.shape[0]):
            for j in range(heatmap.shape[1]):
                # AM/PM
                plt.annotate(f"{i % 12 if i % 12 != 0 else 12}{'AM' if i < 12 else 'PM'}" if heatmap[i, j] != 0 else "", (j+0.5, i+0.5),
                             ha='center', va='center', color='black', fontsize=6, wrap=True)

    plt.show()


# Create date range widgets
start_date_widget = widgets.DatePicker(
    description='Start Date', value=datetime.now().replace(day=1))
end_date_widget = widgets.DatePicker(
    description='End Date', value=data.date.max())
goal_widget = widgets.Text(description='Goal')

show_annotations_widget = widgets.Checkbox(
    value=True, description='Show Annotations')

# Create an interactive widget
interactive_heatmap = widgets.interactive(
    update_heatmap, start_date=start_date_widget, end_date=end_date_widget, goal=goal_widget, show_annotations=show_annotations_widget)
interactive_heatmap

interactive(children=(DatePicker(value=datetime.datetime(2024, 3, 1, 8, 58, 51, 781531), description='Start Da…